In [1]:
def print_bits(lst: list, numcols=8):
    for i in range(0, len(lst)):
        if i%numcols != numcols-1 or i == 0:
            print(lst[i], end=" ")
        else:
            print(lst[i])
    print()
        

In [11]:
def return_round_constants() -> list:
    k = []
    k.extend("0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5".split())
    k.extend("0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174".split())
    k.extend("0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da".split())
    k.extend("0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967".split())
    k.extend("0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85".split())
    k.extend("0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070".split())
    k.extend("0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3".split())
    k.extend("0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2".split())
    for i in range(len(k)):
        k[i] = int(k[i], 16)
    return k

In [3]:
def shift_right(bits, num_shift: int):
    return bits >> num_shift
    '''bits = '0'* num_shift + bits
    bits = bits[0:len(bits)-num_shift]
    return bits'''

for i in range(8):
    print(bin(shift_right(0b10100111, i)))

0b10100111
0b1010011
0b101001
0b10100
0b1010
0b101
0b10
0b1


In [4]:
def rotate_right(bits, num_rotate: int, num_bits: int):
    return (2**num_bits-1)&(bits>>num_rotate|bits<<(num_bits-num_rotate))#(bits >> num_rotate)|(bits << (num_bits - num_rotate)) & 0xFFFFFFFF
    #return bits[len(bits)-num_rotate:] + bits[:len(bits)-num_rotate]

for i in range(9):
    print(bin(rotate_right(0b10100011, i, 8)))

0b10100011
0b11010001
0b11101000
0b1110100
0b111010
0b11101
0b10001110
0b1000111
0b10100011


In [5]:
def bitwise_modulo(bits, mod_num):
    while len(bin(bits))-2 > mod_num:
        bits -= 2**(len(bin(bits))-3)
    return bits

In [27]:
def sha256(entry: str):
    #converts to binary string
    entry = [format(ord(i), '08b') for i in entry]
    entry = "".join(entry)
    
    #pre-processes binary input
    end = bin(len(entry))[2:]
    entry += "1"
    entry += "0"*(512-len(entry)-len(end))
    entry += end
    entry = [entry[x:x+8] for x in range(0, len(entry), 8)]
    print_bits(entry)
    
    #initialize hash values
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h4 = 0x510e527f
    h5 = 0x9b05688c
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19
    
    #initialize round constants
    k = return_round_constants()
    
    #create message schedule
    w = ["".join(entry[i:i+4]) for i in range(0, len(entry), 4)]
    w.extend(["0"*32]*48)
    for i in range(len(w)):
        w[i] = int(w[i], 2)
    #print_bits(message, 2)
    
    for i in range(16, len(w)):
        s0 = rotate_right(w[i-15], 7, 32) ^ rotate_right(w[i-15], 18, 32) ^ shift_right(w[i-15], 3)
        s1 = rotate_right(w[i-2], 17, 32) ^ rotate_right(w[i-2], 19, 32) ^ shift_right(w[i-2], 10)
        w[i] = bitwise_modulo(w[i-16] + s0 + w[i-7] + s1, 32)
    
    print_bits([bin(n) for n in w], 2)
    
    #compression
    a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7
    for i in range(64):
        S1 = rotate_right(e, 6, 32) ^ rotate_right(e, 11, 32) ^ rotate_right(e, 25, 32)
        #print(bin(S1))
        ch = (e & f) ^ ((~ e) & g)
        #print(bin(ch))
        temp1 = bitwise_modulo(h + S1 + ch + k[i] + w[i], 32)
        #print(bin(temp1))
        S0 = rotate_right(a, 2, 32) ^ rotate_right(a, 13, 32) ^ rotate_right(a, 22, 32)
        #print(bin(S0))
        maj = (a & b) ^ (a & c) ^ (b & c)
        #print(bin(maj))
        temp2 = bitwise_modulo(S0 + maj, 32)
        #print(bin(temp2))
        h = g
        g = f
        f = e
        e = bitwise_modulo(d + temp1, 32)
        #print(bin(e))
        d = c
        c = b
        b = a
        a = bitwise_modulo(temp1 + temp2, 32)
        #print(bin(a))
    print()
    answers = [0b01001111010000110100000101010010, 0b11010111111001011000111110000011,
               0b01101000101111110101111101100101, 0b00110101001011011011011011000000,
               0b01110011011101101001110101100100, 0b11011111010011100001100001100010,
               0b01110001000001010001111000000001, 0b10000111000011110000000011010000]
    for var, answer in zip([a, b, c, d, e, f, g, h], answers):
        print(bin(var))
        print(bin(answer))
        print(var == answer)
        
    
sha256("hello world")

01101000 01100101 01101100 01101100 01101111 00100000 01110111 01101111
01110010 01101100 01100100 10000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011000

0b1101000011001010110110001101100 0b1101111001000000111011101101111
0b1110010011011000110010010000000 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b1011000
0b110111010001110000001000110111 0b10000110110100001100000000110001
0b11010011101111010001000100001011 0b1111000001111110100011110000010
0b101010100100000111110011101101 0b1001011001011110111110011001001
0b110001111000011001010001011101 0b100010010011011001001001

In [8]:
#Scratch work
first = 0b01010101
second = 0b11110000
print(bin(first), bin(second))
res = (first<<4) | (second>>4)
print(bin(res))
print(len(str(bin(0b00000000))))
print()
x = "0b10000101000101000101111100110001"
print(bin(int(x, 2)))
print(len("01101010000010011110011001100111"))

0b1010101 0b11110000
0b10101011111
3

0b10000101000101000101111100110001
32
